# Module 2: SQL
## Advanced ML @ SBU (Fall '23)

# Table of Contents
- [Loading Data](#loading-data)
- [SELECT Statement](#select-statement)
    - [LIMIT](#limit)
    - [ORDER BY](#order-by)
- [WHERE Clause](#where-clause)
- [GROUP BY, HAVING and COUNT](#group-by-having-and-count)
    - [COUNT](#count)
    - [GROUP BY](#group-by)
    - [HAVING](#having)
- [WITH](#with)
- [JOINing data, UNIONs](#joining-data-unions)
    - [JOIN](#join)
    - [UNION](#union)
- [EDA with SQL](#eda-with-sql)
- [Storing and Modifying Data](#storing-and-modifying-data)
    - [CREATE TABLE](#create-table)
    - [DROP TABLE](#drop-table)
    - [Database Views](#database-views)
        - [Views vs. Tables](#views-vs-tables)
    - [INSERT INTO](#insert-into)
    - [DELETE FROM](#delete-from)
    - [UPDATE](#update)

# Loading Data

Different protocols require different methods for connecting to a database. Make sure to research your protocol and look up any limitations or specific quirks there might be.

In [81]:
# For MySQL database
# import mysql.connector
# cnx = mysql.connector.connect(user='<username>', password='<password>',
#                               host='<host>', database='<database>')

# For PostgreSQL database
# import psycopg2
# cnx = psycopg2.connect(user='<username>', password='<password>',
#                        host='<host>', port='<port>', database='<database>')

# For SQLite database
import sqlite3

filepath = '.\data\salaries.sqlite'
cnx = sqlite3.connect(filepath)


# SELECT Statement

In SQL, a `SELECT` statement plays a vital role in extracting information from the database. It serves as a powerful tool that, when combined with various SQL keywords, can allow users to efficiently retrieve and manipulate data in a multitude of ways. 

One of its basic functions is to fetch data from specific columns within a database table. Additionally, `SELECT` enables users to merge data from multiple tables, providing a comprehensive view of related data elements, which we'll explore later. When this statement is used in combination with other SQL keywords, this statement can be used for filtering out columns, performing calculations, and much more.

In [102]:
import pandas as pd


# Read data from a table
df = pd.read_sql_query("SELECT * FROM Salaries", con=cnx)

df.head()

,Id,EmployeeName,JobTitle,BasePay,OvertimePay,OtherPay,Benefits,TotalPay,TotalPayBenefits,Year,Notes,Agency,Status
0,1,NATHANIEL FORD,GENERAL MANAGER-METROPOLITAN TRANSIT AUTHORITY,167411.18,0,400184.25,,567595.43,567595.43,2011,,San Francisco,
1,2,GARY JIMENEZ,CAPTAIN III (POLICE DEPARTMENT),155966.02,245131.88,137811.38,,538909.28,538909.28,2011,,San Francisco,
2,3,ALBERT PARDINI,CAPTAIN III (POLICE DEPARTMENT),212739.13,106088.18,16452.6,,335279.91,335279.91,2011,,San Francisco,
3,4,CHRISTOPHER CHONG,WIRE ROPE CABLE MAINTENANCE MECHANIC,77916,56120.71,198306.9,,332343.61,332343.61,2011,,San Francisco,
4,5,PATRICK GARDNER,"DEPUTY CHIEF OF DEPARTMENT,(FIRE DEPARTMENT)",134401.6,9737,182234.59,,326373.19,326373.19,2011,,San Francisco,


In [103]:
df.dtypes

Id                    int64
EmployeeName         object
JobTitle             object
BasePay              object
OvertimePay          object
OtherPay             object
Benefits             object
TotalPay            float64
TotalPayBenefits    float64
Year                  int64
Notes                object
Agency               object
Status               object
dtype: object

In [83]:
df.columns

Index(['Id', 'EmployeeName', 'JobTitle', 'BasePay', 'OvertimePay', 'OtherPay',
       'Benefits', 'TotalPay', 'TotalPayBenefits', 'Year', 'Notes', 'Agency',
       'Status'],
      dtype='object')

In [84]:
df.shape

(148654, 13)

In [85]:
df.describe()

,Id,TotalPay,TotalPayBenefits,Year
count,148654.000000,148654.000000,148654.000000,148654.000000
mean,74327.500000,74768.321972,93692.554811,2012.522643
std,42912.857795,50517.005274,62793.533483,1.117538
min,1.000000,-618.130000,-618.130000,2011.000000
25%,37164.250000,36168.995000,44065.650000,2012.000000
50%,74327.500000,71426.610000,92404.090000,2013.000000
75%,111490.750000,105839.135000,132876.450000,2014.000000
max,148654.000000,567595.430000,567595.430000,2014.000000


In [86]:
df.dtypes

Id                    int64
EmployeeName         object
JobTitle             object
BasePay              object
OvertimePay          object
OtherPay             object
Benefits             object
TotalPay            float64
TotalPayBenefits    float64
Year                  int64
Notes                object
Agency               object
Status               object
dtype: object

SQL `SELECT` queries follow this basic syntax, though most of the clauses are optional:

```
    SELECT [columns to return]
    FROM [schema.table]
    WHERE [conditional filter statements]
    GROUP BY [columns to group on]
    HAVING [conditional filter statements that are run after grouping]
    ORDER BY [columns to sort on]
```

The `SELECT` and `FROM` clauses are generally required, because those indicate which columns to select and from what table. 

In [87]:
# Selecting specific columns
df = pd.read_sql_query("SELECT EmployeeName, Agency, JobTitle FROM Salaries", con=cnx)
df.head()

,EmployeeName,Agency,JobTitle
0,NATHANIEL FORD,San Francisco,GENERAL MANAGER-METROPOLITAN TRANSIT AUTHORITY
1,GARY JIMENEZ,San Francisco,CAPTAIN III (POLICE DEPARTMENT)
2,ALBERT PARDINI,San Francisco,CAPTAIN III (POLICE DEPARTMENT)
3,CHRISTOPHER CHONG,San Francisco,WIRE ROPE CABLE MAINTENANCE MECHANIC
4,PATRICK GARDNER,San Francisco,"DEPUTY CHIEF OF DEPARTMENT,(FIRE DEPARTMENT)"


## Why does it look like this?

We don't *need* to capitalize commands, the following will still work:

In [88]:
df = pd.read_sql_query("selecT EmployeeName, Agency from Salaries", con=cnx)
# Note the "select" and "from" statements above, and how the capitalization differs from the 'norm'.

df.head()

,EmployeeName,Agency
0,NATHANIEL FORD,San Francisco
1,GARY JIMENEZ,San Francisco
2,ALBERT PARDINI,San Francisco
3,CHRISTOPHER CHONG,San Francisco
4,PATRICK GARDNER,San Francisco


SQL writing conventions dictate that we use capital letters for SQL keywords.

### LIMIT

We can also limit the number of results we get, by using the `LIMIT` keyword:

In [89]:
df = pd.read_sql_query("SELECT EmployeeName, Agency FROM Salaries LIMIT 3", con=cnx)
df

,EmployeeName,Agency
0,NATHANIEL FORD,San Francisco
1,GARY JIMENEZ,San Francisco
2,ALBERT PARDINI,San Francisco


# ORDER BY 

The `ORDER BY` clause is used to sort the output rows. 

In it, you list the columns you want to sort the results by, in order, separated by commas. 

You can also specify whether you want the sorting to be in ascending (`ASC`) or descending 
(`DESC`) order. `ASC` sorts text alphabetically and numeric values from low to high, 
and `DESC` sorts them in the reverse order. 

> In MySQL, NULL values appear first when sorting in default ascending order

In [105]:
pd.read_sql_query("SELECT * FROM Salaries ", con=cnx).head()

,Id,EmployeeName,JobTitle,BasePay,OvertimePay,OtherPay,Benefits,TotalPay,TotalPayBenefits,Year,Notes,Agency,Status
0,1,NATHANIEL FORD,GENERAL MANAGER-METROPOLITAN TRANSIT AUTHORITY,167411.18,0,400184.25,,567595.43,567595.43,2011,,San Francisco,
1,2,GARY JIMENEZ,CAPTAIN III (POLICE DEPARTMENT),155966.02,245131.88,137811.38,,538909.28,538909.28,2011,,San Francisco,
2,3,ALBERT PARDINI,CAPTAIN III (POLICE DEPARTMENT),212739.13,106088.18,16452.6,,335279.91,335279.91,2011,,San Francisco,
3,4,CHRISTOPHER CHONG,WIRE ROPE CABLE MAINTENANCE MECHANIC,77916,56120.71,198306.9,,332343.61,332343.61,2011,,San Francisco,
4,5,PATRICK GARDNER,"DEPUTY CHIEF OF DEPARTMENT,(FIRE DEPARTMENT)",134401.6,9737,182234.59,,326373.19,326373.19,2011,,San Francisco,


In [90]:
df = pd.read_sql_query("SELECT EmployeeName, BasePay FROM Salaries ORDER BY TotalPay", con=cnx)

df.head()

,EmployeeName,BasePay
0,Joe Lopez,0
1,PAULETTE ADAMS,0
2,KAUKAB MOHSIN,0
3,JOSEPHINE MCCREARY,0
4,Roland Baylon,0


## Inline Calculations

Let's take another look at our data:

In [91]:
df = pd.read_sql_query("SELECT * FROM Salaries", con=cnx)

df.head()

,Id,EmployeeName,JobTitle,BasePay,OvertimePay,OtherPay,Benefits,TotalPay,TotalPayBenefits,Year,Notes,Agency,Status
0,1,NATHANIEL FORD,GENERAL MANAGER-METROPOLITAN TRANSIT AUTHORITY,167411.18,0,400184.25,,567595.43,567595.43,2011,,San Francisco,
1,2,GARY JIMENEZ,CAPTAIN III (POLICE DEPARTMENT),155966.02,245131.88,137811.38,,538909.28,538909.28,2011,,San Francisco,
2,3,ALBERT PARDINI,CAPTAIN III (POLICE DEPARTMENT),212739.13,106088.18,16452.6,,335279.91,335279.91,2011,,San Francisco,
3,4,CHRISTOPHER CHONG,WIRE ROPE CABLE MAINTENANCE MECHANIC,77916,56120.71,198306.9,,332343.61,332343.61,2011,,San Francisco,
4,5,PATRICK GARDNER,"DEPUTY CHIEF OF DEPARTMENT,(FIRE DEPARTMENT)",134401.6,9737,182234.59,,326373.19,326373.19,2011,,San Francisco,


We can add "temporary" columns that are products of two other columns very easily:

In [92]:
df = pd.read_sql_query("SELECT EmployeeName, BasePay - OtherPay AS Pay FROM Salaries", con=cnx)
df.head()

,EmployeeName,Pay
0,NATHANIEL FORD,-232773.07
1,GARY JIMENEZ,18154.64
2,ALBERT PARDINI,196286.53
3,CHRISTOPHER CHONG,-120390.90
4,PATRICK GARDNER,-47832.99


We can also do some rounding:

In [93]:
df = pd.read_sql_query("SELECT EmployeeName, ROUND(BasePay + OtherPay) as Pay FROM Salaries", con=cnx)
df.head()

,EmployeeName,Pay
0,NATHANIEL FORD,567595.0
1,GARY JIMENEZ,293777.0
2,ALBERT PARDINI,229192.0
3,CHRISTOPHER CHONG,276223.0
4,PATRICK GARDNER,316636.0


# WHERE Clause

The `WHERE` clause is the part of the `SELECT` statement in which you list conditions that are used to determine which rows in the table should be included in the results set. 

> In other words, the `WHERE` clause is used for filtering.

Very similar to conditional statements that use boolean logic to determine what action to take, based on whether 
certain conditions are met. 

SQL uses boolean logic to check the available data against conditions in your `WHERE` clause to determine whether to include each row in the output.

The `WHERE` clause goes after the `FROM` statement and before any `GROUP BY`, `ORDER BY`, or `LIMIT` statements in the `SELECT` query:

```
    SELECT [columns to return]
    FROM [table]
    WHERE [conditional filter statements]
    ORDER BY [columns to sort on]
```

In [94]:
df = pd.read_sql_query("SELECT * FROM Salaries", con=cnx)

In [95]:
# A simple example
df = pd.read_sql_query("SELECT * FROM Salaries WHERE BasePay > 0 ORDER BY BasePay", con=cnx)
df.head()

,Id,EmployeeName,JobTitle,BasePay,OvertimePay,OtherPay,Benefits,TotalPay,TotalPayBenefits,Year,Notes,Agency,Status
0,148620,Ian V Cameron,IS Program Analyst-Assistant,6.04,0,10.05,2.3,16.09,18.39,2014,,San Francisco,PT
1,36088,MORTON BRADLEY,TRACK MAINTENANCE WORKER,14.25,0,56.14,,70.39,70.39,2011,,San Francisco,
2,148621,India C Sabater,Recreation Leader,15.5,0,0,0.16,15.50,15.66,2014,,San Francisco,PT
3,110519,Baltazar J Mata,Public Service Trainee,15.83,0,0,0.16,15.83,15.99,2013,,San Francisco,
4,110520,Leol Aaron Amador,Public Service Trainee,15.83,0,0,0.16,15.83,15.99,2013,,San Francisco,


In [96]:
# Query to show employees with a otherpay + totalpay above 100, ordered by year
query = """SELECT EmployeeName 
FROM Salaries 
WHERE OvertimePay + OtherPay > 100
ORDER BY year 
"""
df = pd.read_sql_query(query, con=cnx)
df.head()

,EmployeeName
0,NATHANIEL FORD
1,GARY JIMENEZ
2,ALBERT PARDINI
3,CHRISTOPHER CHONG
4,PATRICK GARDNER


You can combine multiple conditions with boolean operators, such as `AND`, `OR`, or `AND NOT` between them in order to filter using multiple criteria in the `WHERE` clause.

In [97]:
query = "SELECT Id, EmployeeName, TotalPay FROM Salaries WHERE BasePay < 0 OR TotalPay > 500000 "
df = pd.read_sql_query(query, con=cnx)
df.head()

,Id,EmployeeName,TotalPay
0,1,NATHANIEL FORD,567595.43
1,2,GARY JIMENEZ,538909.28
2,72833,Irwin Sidharta,83.01
3,72866,Robert Scott,61.07
4,72873,Chung Huey Kung,54.61


In [98]:
query = "SELECT * FROM Salaries"
df = pd.read_sql_query(query, con=cnx)
df.columns

Index(['Id', 'EmployeeName', 'JobTitle', 'BasePay', 'OvertimePay', 'OtherPay',
       'Benefits', 'TotalPay', 'TotalPayBenefits', 'Year', 'Notes', 'Agency',
       'Status'],
      dtype='object')

In [99]:
# Query to show employees after the year 2014, with an overtime + otherpay between 100 and 1000, ordered by total pay descending, restrict to 7 rows
query = "SELECT EmployeeName FROM Salaries WHERE Year > 2013 AND (OvertimePay + OtherPay) > 100 AND (OvertimePay + OtherPay)<1000 ORDER BY TotalPay DESC LIMIT 7"
df = pd.read_sql_query(query, con=cnx)
df.head()

,EmployeeName
0,Margaret A Hannaford
1,Susan S Merritt
2,Christopher J Nelson
3,Janice A Papedo
4,Katherine M Mah


To make things easier from here on out, I'll be defining a function that we can pass our SQL query onto, and obtain a pandas DataFrame as a result of our latest query;

In [100]:
def run_query(query):
    return pd.read_sql_query(query, con=cnx)

In [106]:
query = "SELECT * FROM Salaries"
run_query(query)

,Id,EmployeeName,JobTitle,BasePay,OvertimePay,OtherPay,Benefits,TotalPay,TotalPayBenefits,Year,Notes,Agency,Status
0,1,NATHANIEL FORD,GENERAL MANAGER-METROPOLITAN TRANSIT AUTHORITY,167411.18,0,400184.25,,567595.43,567595.43,2011,,San Francisco,
1,2,GARY JIMENEZ,CAPTAIN III (POLICE DEPARTMENT),155966.02,245131.88,137811.38,,538909.28,538909.28,2011,,San Francisco,
2,3,ALBERT PARDINI,CAPTAIN III (POLICE DEPARTMENT),212739.13,106088.18,16452.6,,335279.91,335279.91,2011,,San Francisco,
3,4,CHRISTOPHER CHONG,WIRE ROPE CABLE MAINTENANCE MECHANIC,77916,56120.71,198306.9,,332343.61,332343.61,2011,,San Francisco,
4,5,PATRICK GARDNER,"DEPUTY CHIEF OF DEPARTMENT,(FIRE DEPARTMENT)",134401.6,9737,182234.59,,326373.19,326373.19,2011,,San Francisco,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
148649,148650,Roy I Tillery,Custodian,0,0,0,0,0.00,0.00,2014,,San Francisco,PT
148650,148651,Not provided,Not provided,Not Provided,Not Provided,Not Provided,Not Provided,0.00,0.00,2014,,San Francisco,
148651,148652,Not provided,Not provided,Not Provided,Not Provided,Not Provided,Not Provided,0.00,0.00,2014,,San Francisco,
148652,148653,Not provided,Not provided,Not Provided,Not Provided,Not Provided,Not Provided,0.00,0.00,2014,,San Francisco,


# GROUP BY, HAVING and COUNT

Grouping functions in SQL, such as `GROUP BY`, `HAVING`, and `COUNT` play a crucial role in efficiently aggregating and summarizing data within a database. Aggregation functions in SQL are used to perform calculations on a set of values and return a single value as a result. They are significant because they allow us to analyze and summarize data in a database.


## COUNT

`COUNT` is an aggregate function that counts the number of rows or occurrences meeting specified criteria, making it useful for generating summary reports or identifying patterns in data subsets. These functions find utility in scenarios such as financial analysis, sales forecasting, customer segmentation, and any situation where data needs to be analyzed at a group or category level.

In [107]:
query = """
SELECT COUNT(Id)
FROM Salaries
WHERE BasePay < 0
"""
run_query(query)

,COUNT(Id)
0,11


In [108]:
query = """
SELECT COUNT(Id), EmployeeName
FROM Salaries
WHERE BasePay < 0
"""
run_query(query)

,COUNT(Id),EmployeeName
0,11,Irwin Sidharta


## GROUP BY

The `GROUP BY` clause is used to group rows based on specific columns, allowing for the application of aggregate functions on each group independently. This enables the computation of various metrics like sum, average, count, and maximum/minimum values within each group. 

In [112]:
query = """
SELECT Year, COUNT(Id)
FROM Salaries
WHERE BasePay < 100
GROUP BY Year
"""
run_query(query)

,Year,COUNT(Id)
0,2011,515
1,2012,489
2,2013,61
3,2014,483


In [113]:
query = """
SELECT Year, SUM(BasePay)
FROM Salaries
WHERE BasePay < 100
GROUP BY Year
"""
run_query(query)

,Year,SUM(BasePay)
0,2011,3007.28
1,2012,2648.08
2,2013,3476.57
3,2014,3077.31


Since `GROUP BY` does aggregation, it doesn't make sense to use it without an aggregate function. Therefore, if you have any `GROUP BY` clause, then all variables must be passed to either a:

1. Another `GROUP BY` command,
2. An aggregation function

In [115]:
# The following is useless

query = """
SELECT Year, BasePay
FROM Salaries
GROUP BY Year
"""
run_query(query)

,Year,BasePay
0,2011,167411.18
1,2012,128808.87
2,2013,319275.01
3,2014,129150.01


## HAVING

`HAVING` is used in conjunction with `GROUP BY` to filter groups based on specified conditions, similar to the WHERE clause, but applied to grouped data. It could be thought as the "`WHERE`" to the `GROUP BY` statement in a SQL query.

In [117]:
query = """
SELECT Year, COUNT(Id)
FROM Salaries
WHERE BasePay < 100 
GROUP BY Year
HAVING Year > 2012
"""

query2 = """
SELECT Year, COUNT(Id)
FROM Salaries
WHERE BasePay < 100 AND Year > 2012
GROUP BY Year
"""
run_query(query2)

,Year,COUNT(Id)
0,2013,61
1,2014,483


# WITH


CTEs (Common Table Expressions) in SQL, also known as "WITH" clauses, are temporary named result sets that can be referenced within a SQL statement. They allow you to create subqueries that can be used multiple times within a larger query, providing a way to simplify complex queries and improve readability.

```
WITH cte_name (column1, column2, ...) AS (
    -- subquery
    SELECT ...
    FROM ...
    WHERE ...
)
SELECT ...
FROM ...
```

In [118]:
query = """
SELECT Year, COUNT(Id) AS n_people, MAX(TotalPay) AS pay
    FROM Salaries
    WHERE BasePay < 100
    GROUP BY Year
"""

run_query(query)

,Year,n_people,pay
0,2011,515,142245.22
1,2012,489,184534.66
2,2013,61,310.45
3,2014,483,130320.75


In [120]:
query = """
WITH yearly_results AS (
    SELECT Year, COUNT(Id) AS n_people, MAX(TotalPay) AS pay
    FROM Salaries
    WHERE BasePay < 100
    GROUP BY Year
)
SELECT * 
FROM yearly_results
WHERE n_people > 100
ORDER BY pay
"""

run_query(query)

,Year,n_people,pay
0,2014,483,130320.75
1,2011,515,142245.22
2,2012,489,184534.66


Some key points about CTEs:
1. CTEs are defined using the keyword "WITH" followed by the name of the CTE and optional column names.
2. The CTE is followed by the "AS" keyword and a parenthesized subquery that defines the result set.
3. CTEs can be referenced like a table within the subsequent query.
4. CTEs can be used for recursive queries to handle hierarchical data.
5. CTEs are only visible within the query in which they are defined.
6. CTEs can be self-referencing, allowing the subquery to refer to itself recursively.

Using CTEs can simplify complex queries, improve code reusability, and enhance query performance by optimizing the execution plan.

In [ ]:
run_query("select * from Salaries").columns

In [ ]:
# Exercise: Find the name of the employee with the totalpay for each year

In [ ]:
# Exercise: For each job title, find the highest totalpay for each year

In [ ]:
# Exercise: Changes in the number of values for the status column on a year-by-year basis?

In [ ]:
# Exercise: Using seaborn or matplotlib, based on the Status column, illustrate the difference in totalpay (scatterplot)

# Joining data, UNIONs



> For this part, we'll be working with a larger dataset. The one we'll be working with is the [World Development Indicators 2022](https://www.kaggle.com/datasets/psycon/world-development-indicators?select=indicators.sqlite) hosted on Kaggle.

First, we'll load this larger dataset;

In [121]:
# Load larger data

filepath = '.\data\indicators.sqlite'
cnx_indi = sqlite3.connect(filepath)

In [122]:
def run_query_indi(query):
    return pd.read_sql_query(query, con=cnx_indi)

In [123]:
# Preview one of the tables

query = """
SELECT * 
FROM Country
"""
run_query_indi(query).head()

,CountryCode,ShortName,TableName,LongName,Alpha2Code,CurrencyUnit,SpecialNotes,Region,IncomeGroup,Wb2Code,...,GovernmentAccountingConcept,ImfDataDisseminationStandard,LatestPopulationCensus,LatestHouseholdSurvey,SourceOfMostRecentIncomeAndExpenditureData,VitalRegistrationComplete,LatestAgriculturalCensus,LatestIndustrialData,LatestTradeData,LatestWaterWithdrawalData
0,AFG,Afghanistan,Afghanistan,Islamic State of Afghanistan,AF,Afghan afghani,Fiscal year end: March 20; reporting period fo...,South Asia,Low income,AF,...,Consolidated central government,General Data Dissemination System (GDDS),1979,"Multiple Indicator Cluster Survey (MICS), 2010/11","Integrated household survey (IHS), 2008",,2013/14,,2013,2000
1,ALB,Albania,Albania,Republic of Albania,AL,Albanian lek,,Europe & Central Asia,Upper middle income,AL,...,Budgetary central government,General Data Dissemination System (GDDS),2011,"Demographic and Health Survey (DHS), 2008/09",Living Standards Measurement Study Survey (LSM...,Yes,2012,2011,2013,2006
2,DZA,Algeria,Algeria,People's Democratic Republic of Algeria,DZ,Algerian dinar,,Middle East & North Africa,Upper middle income,DZ,...,Budgetary central government,General Data Dissemination System (GDDS),2008,"Multiple Indicator Cluster Survey (MICS), 2012","Integrated household survey (IHS), 1995",,,2010,2013,2001
3,ASM,American Samoa,American Samoa,American Samoa,AS,U.S. dollar,,East Asia & Pacific,Upper middle income,AS,...,,,2010,,,Yes,2007,,,
4,ADO,Andorra,Andorra,Principality of Andorra,AD,Euro,,Europe & Central Asia,High income: nonOECD,AD,...,,,2011. Population data compiled from administra...,,,Yes,,,2006,


In [124]:
query = """
SELECT * 
FROM Country
WHERE CountryCode == 'ALB'
"""

run_query_indi(query).head()

,CountryCode,ShortName,TableName,LongName,Alpha2Code,CurrencyUnit,SpecialNotes,Region,IncomeGroup,Wb2Code,...,GovernmentAccountingConcept,ImfDataDisseminationStandard,LatestPopulationCensus,LatestHouseholdSurvey,SourceOfMostRecentIncomeAndExpenditureData,VitalRegistrationComplete,LatestAgriculturalCensus,LatestIndustrialData,LatestTradeData,LatestWaterWithdrawalData
0,ALB,Albania,Albania,Republic of Albania,AL,Albanian lek,,Europe & Central Asia,Upper middle income,AL,...,Budgetary central government,General Data Dissemination System (GDDS),2011,"Demographic and Health Survey (DHS), 2008/09",Living Standards Measurement Study Survey (LSM...,Yes,2012,2011,2013,2006


In [125]:
query = """
SELECT * 
FROM Indicators
WHERE CountryCode == 'ARB'
"""
run_query_indi(query).head()

,CountryName,CountryCode,IndicatorName,IndicatorCode,Year,Value
0,Arab World,ARB,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,1960,1.335609e+02
1,Arab World,ARB,Age dependency ratio (% of working-age populat...,SP.POP.DPND,1960,8.779760e+01
2,Arab World,ARB,"Age dependency ratio, old (% of working-age po...",SP.POP.DPND.OL,1960,6.634579e+00
3,Arab World,ARB,"Age dependency ratio, young (% of working-age ...",SP.POP.DPND.YG,1960,8.102333e+01
4,Arab World,ARB,Arms exports (SIPRI trend indicator values),MS.MIL.XPRT.KD,1960,3.000000e+06


## JOIN

So as you can tell, there are columns with matching values between the two tables. We can join these tables together using these shared values to make a singular table that we can work with;

In [128]:
query = """
SELECT CountryName, Country.CountryCode, IndicatorName, CurrencyUnit
FROM Country
INNER JOIN Indicators
    ON Country.CountryCode = Indicators.CountryCode
"""

run_query_indi(query)

,CountryName,CountryCode,IndicatorName,CurrencyUnit
0,Afghanistan,AFG,"Adolescent fertility rate (births per 1,000 wo...",Afghan afghani
1,Afghanistan,AFG,Age dependency ratio (% of working-age populat...,Afghan afghani
2,Afghanistan,AFG,"Age dependency ratio, old (% of working-age po...",Afghan afghani
3,Afghanistan,AFG,"Age dependency ratio, young (% of working-age ...",Afghan afghani
4,Afghanistan,AFG,Arms imports (SIPRI trend indicator values),Afghan afghani
...,...,...,...,...
5656453,Zimbabwe,ZWE,Time required to register property (days),U.S. dollar
5656454,Zimbabwe,ZWE,Time required to start a business (days),U.S. dollar
5656455,Zimbabwe,ZWE,Time to prepare and pay taxes (hours),U.S. dollar
5656456,Zimbabwe,ZWE,Time to resolve insolvency (years),U.S. dollar


We can also use `WITH` clauses to perform more complex operations;

In [131]:
query = """
WITH  country_currency AS (
SELECT CountryName, Country.CountryCode, IndicatorName, CurrencyUnit
FROM Country
INNER JOIN Indicators
    ON Country.CountryCode = Indicators.CountryCode
)
SELECT CountryName
FROM country_currency
WHERE CurrencyUnit == 'U.S. dollar'
GROUP BY CountryName
"""

run_query_indi(query)

,CountryName
0,American Samoa
1,Ecuador
2,El Salvador
3,Guam
4,Liberia
5,Marshall Islands
6,"Micronesia, Fed. Sts."
7,Northern Mariana Islands
8,Palau
9,Puerto Rico


In [132]:
run_query_indi("SELECT * FROM Country").columns

Index(['CountryCode', 'ShortName', 'TableName', 'LongName', 'Alpha2Code',
       'CurrencyUnit', 'SpecialNotes', 'Region', 'IncomeGroup', 'Wb2Code',
       'NationalAccountsBaseYear', 'NationalAccountsReferenceYear',
       'SnaPriceValuation', 'LendingCategory', 'OtherGroups',
       'SystemOfNationalAccounts', 'AlternativeConversionFactor',
       'PppSurveyYear', 'BalanceOfPaymentsManualInUse',
       'ExternalDebtReportingStatus', 'SystemOfTrade',
       'GovernmentAccountingConcept', 'ImfDataDisseminationStandard',
       'LatestPopulationCensus', 'LatestHouseholdSurvey',
       'SourceOfMostRecentIncomeAndExpenditureData',
       'VitalRegistrationComplete', 'LatestAgriculturalCensus',
       'LatestIndustrialData', 'LatestTradeData', 'LatestWaterWithdrawalData'],
      dtype='object')

In [133]:
run_query_indi("SELECT * FROM Indicators").columns

Index(['CountryName', 'CountryCode', 'IndicatorName', 'IndicatorCode', 'Year',
       'Value'],
      dtype='object')

In [ ]:
# Exercise: Using joins, for each Region, find the name of the maximum indicator value

query = """

"""
run_query_indi(query)

## UNION

While `JOIN`s perform horizontal concatenation, `UNION`s perform vertical concatenation;

In [134]:
query= """
SELECT CountryCode
FROM Country
UNION ALL
SELECT CountryCode
FROM Indicators
"""

run_query_indi(query)

,CountryCode
0,AFG
1,ALB
2,DZA
3,ASM
4,ADO
...,...
5656700,ZWE
5656701,ZWE
5656702,ZWE
5656703,ZWE


# EDA with SQL

In [ ]:
# Later! (Homework)

# Storing and Modifying Data

In most databases, you can store the results of a query as either a table or a view. 

Storing results as a table takes a snapshot of whatever the results are at the time 
the query is run and saves the data returned as a new table object, or as new 
rows appended to an existing table, depending on how you write your SQL 
statement. A database view instead stores the SQL itself and runs it on-demand 
when you write a query that references the name of the view, to dynamically generate a new dataset based on the state of the referenced database objects at 
the time you run the query. 

## CREATE TABLE

One way to store the results of a query is to use a CREATE TABLE statement. 
The syntax is

```
CREATE TABLE [schema_name].[new_table_name] AS
(
 [your query here]
)
```

So, for instance:

In [ ]:
# This won't work for silly reasons, but syntax is correct 

query = """
CREATE TABLE max_region_indi AS 
    SELECT Region, IndicatorName, MAX(Value)
    FROM Country
    INNER JOIN Indicators ON Country.CountryCode == Indicators.CountryCode
    GROUP BY Region
"""

run_query_indi(query)

In [ ]:
run_query_indi("SELECT * FROM max_region_indi")

## DROP TABLE

The syntax for dropping a table is simply:

```
DROP TABLE [schema_name].[table_name]
```

In [ ]:
query = "DROP TABLE max_region_indi"
run_query_indi(query)

### Views vs. Tables

Database views are created and dropped the same exact way as tables, though 
when you create a view, you are not actually storing the data, but storing the 
query to be run when you query the view. So when you drop a view, you are 
not actually deleting any data, since the data isn’t stored; you are just dropping 
the named reference to the query:

```
CREATE VIEW product_units_vw AS
(
 SELECT * 
 FROM farmers_market.product 
 WHERE product_qty_type = "unit"
)

SELECT * FROM farmers_market.product_units_vw

DROP VIEW farmers_market.product_units_vw
```